# Image classification through fine-tuning
In this example, we will perform fine-tuning, which allows you to classify images even when the available dataset is not large enough. The idea is to exploit networks already trained on very large datasets to extract low-level features, and learn only the weights of the layers that are relative to high-level features. It is a very simple form of *transfer learning* which is still very effective.
We will apply this strategy to image classification and we will use **keras_hub** and **easy_cv_dataset** libraries that can be installed with the followinh instruction:

In [ ]:
!pip install -q --upgrade keras_hub git+https://github.com/davin11/easy-cv-dataset

Now, we will import the stantard libraries, **keras_hub**, and **easy_cv_dataset** (with the alias ds).

In [ ]:
%reset -f
import numpy as np
import matplotlib.pyplot as plt
import skimage.io as io
import keras
import keras_hub
import easy_cv_dataset as ds

## Data preparation
We will use the dataset TrafficSigns, therefore the objective is to identify whether in the image there is a road sign of the type: mandatory, prohibition or warning. On Notebook you can directly execute the following instructions to download and decompress the training set:

In [ ]:
!wget -q -c https://www.grip.unina.it/download/guide_TF/TrafficSigns.zip
!unzip -q -n TrafficSigns.zip

You will find a folder called "TrafficSigns" which contains two sub-folders "train" and "test". Both the "train" and "test" folders contain the "mandatory", "prohibition" and "warning" folders. Display an image for each traffic signwith the following code:

In [ ]:
img0 = io.imread('TrafficSigns/train/mandatory/img0065_00.png')
img1 = io.imread('TrafficSigns/train/prohibition/img2929_00.png')
img2 = io.imread('TrafficSigns/train/warning/img2470_00.png')

plt.figure(figsize=(15,5))
plt.subplot(1,3,1); plt.imshow(img0); plt.title('mandatory')
plt.subplot(1,3,2); plt.imshow(img1); plt.title('prohibition')
plt.subplot(1,3,3); plt.imshow(img2); plt.title('warning')
plt.show()

We proceed to prepare the data using the functions of `ds.image_dataframe_from_directory`,
which simplify this operation. Since the images are organized into folders, each of which is associated with a
class, we can obtain a list of images with their respective classes by specifying only the parent folder.

In [ ]:
train_table = ds.image_dataframe_from_directory('TrafficSigns/train')

The previous instruction creates a table, called `train_table`, which has two columns: the image column containing the file paths of the training images, and the class column containing the class information for each image. To display the first rows of the table, execute the following instruction:

In [ ]:
display(train_table)

Let's divide the training table in two tables for training and validation, respectively using the function `train_test_split` of `sklearn.model_selection`:

In [ ]:
from sklearn.model_selection import train_test_split
train_table, valid_table = train_test_split(train_table, test_size=0.2,
                                            random_state=34,
                                            stratify=train_table['class'])

We need to indicate the parameter test size to set the percentage of the validation data to 20%, while the
parameter `stratify=train table['class']` is used t o guarantee a balanced division among the classes.
By using the function `ds.image_classification_dataset_from_dataframe`, we can prepare the training set
images.

In [ ]:
batch_size = 8
img_height, img_width = 224, 224

train_dataset = ds.image_classification_dataset_from_dataframe(
  train_table, batch_size=batch_size, shuffle=True,
  pre_batching_processing=keras.layers.Resizing(img_height, img_width),
  post_batching_processing=None,
  do_normalization=False,
  class_mode='categorical')

The function `ds.image_classification_dataset_from_dataframe` requires the previously created table as its first parameter.
The second parameter, `batch_size`, indicates the size of the batch that will be used during training and has to be specified to determine how many images should be processed together.
The `shuffle` parameter indicates whether the dataset should be shuffled at the beginning of each epoch.
The `pre_batching_processing` and `post_batching_processing` parameters allow for applying an operation to all images in the dataset before and after the batch construction, respectively. When dealing with images of different resolutions, we can specify a resizing operation before batching to resize the images to a given dimension.
The parameter `do_normalization` can be used to normalize the images to range [0, 1].
It is set `False` because the normalizztion is made by the model in this examples.
The last parameter, `class_mode`, is used to indicate the format of the labels. By using the string `'categorical'` the OneHot format will be utilized.
Let's also prepare the validation dataset.

In [ ]:
valid_dataset = ds.image_classification_dataset_from_dataframe(
  valid_table, batch_size=batch_size, shuffle=False,
  pre_batching_processing=keras.layers.Resizing(img_height, img_width),
  do_normalization=False,
  class_mode='categorical')

## Network definition
Fine-tuning consists in training a network not starting from random weights,
but using the weights already obtained from a pre-training on another dataset (much larger than the one available). In particular, we use the weights relative to the first layers that extract features common to many images (low-level features) and retrain subsequent layers, that extract specific features of the specific application.
Let's define ResNet50 backbone pre-trained on ImageNet using **keras_hub** library:

In [ ]:
from keras_hub.models import Backbone

pretrained_model = 'resnet_50_imagenet'
backbone = Backbone.from_preset(pretrained_model)

The **Backbone** is the network architecure without the last head block.
The function `Backbone.from_preset` creates the architecture and loads the weights specifying the pretrained model.
A list of available pretrained models is at the web page https://keras.io/keras_hub/presets/

During pre training, each model use a specific image preparation, we need to create a layer that makes the same preparation.

In [ ]:
from keras_hub.layers import ImageConverter
normalization = ImageConverter.from_preset(pretrained_model, image_size=(img_height, img_width))

We then define a model for image classification that includes the backbone and subsequently performs a GlobalAveragePooling and a Dense layer.

In [ ]:
from keras_hub.models import ImageClassifier, ImageClassifierPreprocessor
model = ImageClassifier(
    preprocessor=ImageClassifierPreprocessor(normalization),
    backbone=backbone,
    pooling="avg",          # GlobalAveragePooling layer
    num_classes=3,          # units of Dense layer
    activation="softmax",   # activation function of Dense layer
)
model.summary()

Not training the first layers reduces the parameters to be learned and also the risk of over-fitting. To lock the parameters of the first 25 layers of ResNet50 use the following code:

In [ ]:
train_after_layer = 25
for layer in model.backbone.layers[:train_after_layer]:
    layer.trainable = False
model.summary()

## Training
We define the optimizer and the loss function: 

In [ ]:
model.compile(loss=keras.losses.CategoricalCrossentropy(from_logits=False),
             optimizer=keras.optimizers.SGD(learning_rate=1e-4, momentum=0.9),
             metrics=[keras.metrics.CategoricalAccuracy(), ])

We can also use a very low learning rate since we do not start from random weights. We need to use the fit method for training:

In [ ]:
model.fit(train_dataset,  validation_data=valid_dataset, epochs=5, verbose=True)

Note that with only five epoch we are able to obtain an accuracy on validation data greater than 85%.

## Evaluation
Use the functions `ds.image_dataframe_from_directory` and `ds.image_classification_dataset_from_dataframe` to prepare the images for the test set.

In [ ]:
test_table = ds.image_dataframe_from_directory('TrafficSigns/test')
test_dataset = ds.image_classification_dataset_from_dataframe(
    test_table, batch_size=batch_size, shuffle=False,
    pre_batching_processing=keras.layers.Resizing(img_height, img_width),
    do_normalization=False,
    class_mode='categorical')

To evaluate the performance on the test set then run the following code:

In [ ]:
test_loss, test_accuracy = model.evaluate(test_dataset, verbose=True)
print('Test loss:', test_loss)
print('Test accuracy:', test_accuracy)

## Data Augmentation
We can apply various random operations to the training images, allowing us to artificially increase the size of the training dataset through data augmentation. For this purpose, we can utilize the random operations defined in the `keras.layers` preprocessing library. Additionally, we can combine these operations together using `keras.layers.Pipeline`:

In [ ]:
from keras.layers import RandomBrightness, RandomZoom, Pipeline
augmenter = Pipeline(layers=[
  RandomBrightness(factor=(-0.1, 0.1), value_range=(0, 255)),
  RandomZoom((-0.2,0.2)),
])

train_dataset = ds.image_classification_dataset_from_dataframe(
          train_table, batch_size=batch_size, shuffle=True,
          pre_batching_processing=keras.layers.Resizing(img_height, img_width),
          post_batching_processing=augmenter,
          do_normalization=False,
          class_mode='categorical')